# Exploration of Amazon consumer products reviews
## by Kyle McMillan

## Preliminary Wrangling

> This investigation explores the Amazon US marketplace reviews dataset containing approximately 7 million unique reviews.

In [92]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
import random
random.seed(42)

%matplotlib inline

In [93]:
#Data is loaded from a TSV file. Bad lines are dropped as fixing these manually will take too much time 
#for the low number of lines with errors.
reviews=pd.read_csv(r"amazon_reviews_multilingual_US_v1_00.tsv", sep="\t", error_bad_lines=False, header=0)

b'Skipping line 3231472: expected 15 fields, saw 22\n'
b'Skipping line 3509762: expected 15 fields, saw 22\n'
b'Skipping line 4018793: expected 15 fields, saw 22\n'
b'Skipping line 4280173: expected 15 fields, saw 22\nSkipping line 4290596: expected 15 fields, saw 22\n'
b'Skipping line 4331421: expected 15 fields, saw 22\nSkipping line 4340267: expected 15 fields, saw 22\nSkipping line 4341665: expected 15 fields, saw 22\nSkipping line 4386155: expected 15 fields, saw 22\nSkipping line 4388098: expected 15 fields, saw 22\n'
b'Skipping line 4408027: expected 15 fields, saw 22\nSkipping line 4442615: expected 15 fields, saw 22\n'
b'Skipping line 4519623: expected 15 fields, saw 22\n'
b'Skipping line 4525797: expected 15 fields, saw 22\nSkipping line 4543519: expected 15 fields, saw 22\n'
b'Skipping line 4587726: expected 15 fields, saw 22\nSkipping line 4589301: expected 15 fields, saw 22\nSkipping line 4634393: expected 15 fields, saw 22\n'
b'Skipping line 4666168: expected 15 fields, s

In [94]:
#View the unique items in the product category.
reviews.product_category.unique()

array(['Books', 'Music', 'Video', 'Video DVD', 'Toys', 'Tools',
       'Office Products', 'Video Games', 'Software',
       'Digital_Music_Purchase', 'Home Entertainment', 'Electronics',
       'Digital_Ebook_Purchase', 'Digital_Video_Download', 'Kitchen',
       'Camera', 'Outdoors', 'Musical Instruments', 'Sports', 'Watches',
       'PC', 'Home', 'Wireless', 'Beauty', 'Baby', 'Home Improvement',
       'Apparel', 'Shoes', 'Lawn and Garden', 'Mobile_Electronics',
       'Health & Personal Care', 'Grocery', 'Luggage',
       'Personal_Care_Appliances', 'Automotive', 'Mobile_Apps',
       'Furniture', '2012-12-22', 'Pet Products'], dtype=object)

In [95]:
#View the data that has a product category of a date.
reviews.query("product_category == '2012-12-22'")

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
1852794,US,49422747,R2T9JZNQ996WRC,1568652240,569473707,Emma (Large Print)\tBooks\t5\t0\t0\tN\tY\tFoll...,2012-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
#View the data string in the product title
list(reviews[reviews.review_id=="R2T9JZNQ996WRC"].product_title)

['Emma (Large Print)\tBooks\t5\t0\t0\tN\tY\tFollows Austen\'s book closley\tI have the movie with Ms Paltrow starring, so I bought the book to see how it matched. Very well done.\t2012-12-22\nUS\t7209000\tR1MZOZGBLVKYVP\tB003E8P9G0\t446279348\tThe Kane Chronicles, Book One: The Red Pyramid\tDigital_Ebook_Purchase\t5\t0\t0\tN\tN\tso....\tmy brother read this book and he was instantly addicted to it so i started reading it on my kindle and i goten adddicted to it too. i would say that it is a must read.Questions or Comments my kindle email is  haili@kindle.com\t2012-12-22\nUS\t49422747\tR31UI3EECPWNVA\tB002EWD0I6\t647475881\tLark Rise to Candleford: Season 1\tVideo DVD\t4\t0\t0\tN\tY\tGood\tI thought this would be more of theprevious characters of Cranford, but not to be.  This was good enough for me to order the 2nd season.\t2012-12-22\nUS\t7209000\tR2UXMYNK8AEEVR\tB005CRQ4GU\t632021160\tThe Third Wheel (Diary of a Wimpy Kid, Book 7)\tDigital_Ebook_Purchase\t5\t1\t2\tN\tN\tso....\tI LOV

In [97]:
#Drop this line as it appears to have been parsed wrong because of the extra "/t" in the title column and there are a lot
#of extra reviews in this title string.
reviews.drop(reviews.loc[reviews.product_category.isin(['2012-12-22'])].index, inplace=True)
reviews.product_category.unique()

array(['Books', 'Music', 'Video', 'Video DVD', 'Toys', 'Tools',
       'Office Products', 'Video Games', 'Software',
       'Digital_Music_Purchase', 'Home Entertainment', 'Electronics',
       'Digital_Ebook_Purchase', 'Digital_Video_Download', 'Kitchen',
       'Camera', 'Outdoors', 'Musical Instruments', 'Sports', 'Watches',
       'PC', 'Home', 'Wireless', 'Beauty', 'Baby', 'Home Improvement',
       'Apparel', 'Shoes', 'Lawn and Garden', 'Mobile_Electronics',
       'Health & Personal Care', 'Grocery', 'Luggage',
       'Personal_Care_Appliances', 'Automotive', 'Mobile_Apps',
       'Furniture', 'Pet Products'], dtype=object)

In [98]:
print(reviews.info(null_counts=True ))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6900885 entries, 0 to 6900885
Data columns (total 15 columns):
marketplace          6900885 non-null object
customer_id          6900885 non-null int64
review_id            6900885 non-null object
product_id           6900885 non-null object
product_parent       6900885 non-null int64
product_title        6900885 non-null object
product_category     6900885 non-null object
star_rating          6900885 non-null float64
helpful_votes        6900885 non-null float64
total_votes          6900885 non-null float64
vine                 6900885 non-null object
verified_purchase    6900885 non-null object
review_headline      6900811 non-null object
review_body          6900810 non-null object
review_date          6900562 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 842.4+ MB
None


In [99]:
#Investigate the reason that a few reviews have NaN for a date
dateless = reviews.loc[reviews['review_date'].isin(reviews.review_date.dropna().unique())==False]
dateless.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
142642,US,52767614,RJBEAQJ92LS61,B00003CXDG,765637830,Mission: Impossible 2 (Widescreen Edition),Video DVD,2.0,0.0,0.0,N,N,Technology is not always beneficial -\tespecia...,2001-03-08,NaN
186564,US,50374272,R2LXX6V7B1PICZ,B00005OB0A,321992753,Fever,Music,5.0,0.0,0.0,N,N,The music you were playing really blew my mind...,2001-10-25,NaN
217791,US,42627253,R1CRCYSML85MB4,B00005JKZQ,167276143,Showtime,Video DVD,5.0,5.0,6.0,N,N,It's............SHOWTIME!!!!!\tShowtime is the...,2002-03-16,NaN
235044,US,52442862,R1YZIXBH6AVF4E,B00000JMQC,79142420,Return to Oz [VHS],Video,5.0,6.0,7.0,N,N,The Wizard of Oz it is not--should not be!\tFo...,2002-06-10,NaN
239621,US,47692344,R12KVFMZYPXNF6,B00003CWL6,304141589,American Beauty (1999),Video DVD,5.0,1.0,3.0,N,N,"It's just STUFF!!!\t\\American Beauty\\"" is in...",2002-06-30,NaN


In [100]:
#It seems there was an issue during the reading of the TSV file and the review titles used a tab space. Where read_csv was
#looking for a "\t" delimiter

#Investigate the titles for each of the reviews to see if there is any thing unusual
#As this is a very large and long list, a random sample of 5 were chosen.
DL_random_list = random.sample(list(dateless.review_id), 5)
for i in DL_random_list:
    print(re.split("\t", list(dateless[dateless.review_id==i].review_headline)[0])[0])

The Lone survivor
My teacher said I'm a loser, I give a f**k if you feel me..
Love the boys,  but not this movie
What did you expect?.....Lord Tennyson?
Oldie, but goodie


In [101]:
#Investigate the body of the dateless reviews to see if there is anything unusual.
for i in DL_random_list:
    print(re.split("\t", list(dateless[dateless.review_id==i].review_headline)[0])[1:])

["[[ASIN:0316067601 Lone Survivor: The Eyewitness Account of Operation Redwing and the Lost Heroes of SEAL Team 10]]<br /><br />Excellent narration. Inspiring, patriotic, and courageous. Also, one lucky SEAL. As an American, I am honored, and humbled by his 'call to service\\\\.<br />Bravo."]
['.. I\'m gonna follow my heart.\\\\ <BR>My favorite line on this album off of the highest point on this album, \\\\"Get Em High,\\\\" featuring epic rappers Talib Kweli and Common. I loved it because it influences me and inspires me, but after learning that this was the genius that produced five star tracks such as Poppa Was A Playa by Nas and Izzo by Jay-Z, I wanted to pick up this solely for the beats. I wasn\'t as interested in hearing him as a rapper, because I hadn\'t even heard Through the Wire or All Falls Down at the time. But, I wasn\'t disappointed at all, this guy is one of the best \\\\"new artist\\\\" rappers I\'ve heard in a long time. This guy knows how to write intellectual flows 

In [102]:
#It seems like there are a lot of other reviews mixed into the body of some of the dateless reviews.
#As there are a very large number of total reviews, and wrangling the data to sort out the 300 or so that need to be fixed,
#I have decided to drop these rows.
reviews.drop(dateless.index, inplace=True)
reviews.loc[reviews['review_date'].isin(reviews.review_date.dropna().unique())==False]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date


In [103]:
#Investigate the reviews where there is no headline to see if there is anything unusual.
reviews.loc[reviews['review_headline'].isin(reviews.review_headline.dropna().unique())==False]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
144995,US,51263804,RRR6EK045G5YL,630438551X,212477251,Romeo & Juliet [VHS],Video,5.0,4.0,8.0,N,N,NaN,I consider myself to be a pretty much diehard ...,2001-03-21
274671,US,34490235,R25MC0QXPV5CRZ,6305949980,664817538,The Nightmare Before Christmas (Special Edition),Video DVD,4.0,1.0,6.0,N,N,NaN,"Three words, \\""BEST MOVIE EVER.\\"" If you hav...",2002-12-11
373447,US,23532323,R1MNPI81JJ62NI,B0000DD7NL,868764779,The Diary of Alicia Keys,Music,5.0,0.0,2.0,N,N,NaN,I love this cd! My favorite two songs are:<br...,2004-01-12
450489,US,43058957,RIHK7J9KBB62V,B00005V3Z4,681790048,Donnie Darko (Widescreen Edition),Video DVD,4.0,3.0,8.0,N,N,NaN,It actually is pretty hard to classify this mo...,2004-11-18
515646,US,34215909,R1HAKDWL5X86B9,B00014NE62,59241661,Maurice - The Merchant Ivory Collection,Video DVD,4.0,2.0,3.0,N,N,NaN,"Goes at quite a steady pace, however, this is ...",2005-08-12
522262,US,36882963,R2VEG37XSHHQMO,B0002Y4TTC,773850099,Blind Guardian - Imaginations Through the Look...,Video DVD,3.0,1.0,8.0,N,N,NaN,This is a beautifully packaged 2-disc DVD edit...,2005-09-04
546262,US,24610674,R3NODAYWJSH67Y,B000BM6AVA,828472430,Hypnotize,Music,5.0,2.0,4.0,N,N,NaN,"It's a warm, peaceful afternoon in the upstand...",2005-11-28
550148,US,16196376,R1DUI7MJ3FPONG,B0001YRVN4,745272701,Star Wars Trilogy (A New Hope / The Empire Str...,Video DVD,5.0,4.0,10.0,N,N,NaN,What's all this I hear about people complainin...,2005-12-12
567259,US,22947656,R1ABIC2ULD2TCB,B000AP2ZDK,577609284,Donuts,Music,5.0,112.0,124.0,N,N,NaN,"This isn't Dilla's best work, far from it real...",2006-02-16
568389,US,42224700,R3QMAWGAVYTR9F,B00001U0E1,475263130,Shakespeare in Love (Miramax Collector's Series),Video DVD,4.0,0.0,1.0,N,N,NaN,"I am a Shakespeare buff, so I didn't find this...",2006-02-20


In [104]:
#Investigate the reviews where there is no body to see if there is anything unusual.
reviews.loc[reviews['review_body'].isin(reviews.review_body.dropna().unique())==False]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
4362503,US,44409249,R28HX7V41NGPGD,B00IFMHZ58,731881208,Veronica Mars,Digital_Video_Download,4.0,0.0,0.0,N,Y,Four Stars,NaN,2014-07-14
4364915,US,34483539,R1KRVATE3RM685,0545615402,160159764,Hogwarts Library (Harry Potter),Books,5.0,0.0,1.0,N,Y,Five Stars,NaN,2014-07-14
4492736,US,14326822,R3QYW87GDRRNAF,B004S82OAE,235122690,CCNA Cisco Certified Network Associate Study G...,Digital_Ebook_Purchase,5.0,0.0,1.0,N,Y,Five Stars,NaN,2014-08-03
4646674,US,20596625,R12WMFWQ96MK9E,B00HPYMVD8,108613172,The Target (Will Robie),Digital_Ebook_Purchase,4.0,0.0,0.0,N,Y,Four Stars,NaN,2014-08-29
4867197,US,11236181,R1ZS0RW3SLED36,B0096YJDNQ,454110113,Distant Suns (max) - Unleash your inner astron...,Mobile_Apps,5.0,0.0,0.0,N,Y,Five Stars,NaN,2014-10-07
4967822,US,3152188,R1RGMJWY7KCMFF,B003MYYJD0,120446899,Invicta Men's 6981 Pro Diver Analog Swiss Chro...,Watches,5.0,94.0,102.0,N,Y,Five Stars,NaN,2014-10-24
4990204,US,8149238,R221BRF51GRMNT,B004GJDQT8,36653526,Amazon Underground,Mobile_Apps,4.0,5.0,6.0,N,Y,Four Stars,NaN,2014-10-28
5155825,US,33444040,R2XT5Y4GSMFGBJ,B000LXQVA4,535123469,Fisher-Price Rainforest Jumperoo,Baby,5.0,0.0,0.0,N,N,Five Stars,NaN,2014-11-25
5166126,US,44256154,RK1UNP8GPEEKV,B002IIY6X4,795348432,Homesick,Music,5.0,4.0,4.0,N,Y,Five Stars,NaN,2014-11-27
5190776,US,32311114,R1Q33KCFX6EZKX,B001XVD21Y,818712447,The Departed,Digital_Video_Download,5.0,8.0,9.0,N,Y,Five Stars,NaN,2014-11-30


Looking at the 2 sub tables of the bodyless and titleless reviews, it seems like there were no parsing errors with these but rather people simply did not write anything in these fields when writing their review.  
As such, I will keep these as they are and not remove them.  

There were a few lines that were dropped, these were to save time with this dataset. In total there are around 7 million reviews in this data set, and some need to be fixed manually. These rows of data of approximatley 1000 reviews were dropped as the final table has slightly more than 6.9Million rows.

In [105]:
#As this dataset is only for the US marketplace and all of these values are "US" this column is dropped.
reviews.drop("marketplace", inplace=True, axis=1)

In [106]:
#Check for duplicate rows.
sum(reviews.duplicated())

0

In [107]:
#Convert columns to the correct data type.
reviews['review_date']=pd.to_datetime(reviews.review_date)
reviews['vine'] = reviews.vine.map({'N':False, 'Y':True}).astype(bool)
reviews['star_rating']=pd.to_numeric(reviews.star_rating, downcast="integer")
reviews['helpful_votes']=pd.to_numeric(reviews.helpful_votes, downcast="integer")
reviews['total_votes']=pd.to_numeric(reviews.total_votes, downcast="integer")
reviews['verified_purchase'] = reviews.verified_purchase.map({'N':False, 'Y':True}).astype(bool)
reviews['customer_id'] = reviews.customer_id.astype(str)
reviews['product_parent'] = reviews.product_parent.astype(str)

In [108]:
#High level overview of the data shape and composition.
print(reviews.shape)
print(reviews.info(null_counts=True ))
print(reviews.head(10))

(6900562, 14)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6900562 entries, 0 to 6900885
Data columns (total 14 columns):
customer_id          6900562 non-null object
review_id            6900562 non-null object
product_id           6900562 non-null object
product_parent       6900562 non-null object
product_title        6900562 non-null object
product_category     6900562 non-null object
star_rating          6900562 non-null int8
helpful_votes        6900562 non-null int16
total_votes          6900562 non-null int16
vine                 6900562 non-null bool
verified_purchase    6900562 non-null bool
review_headline      6900488 non-null object
review_body          6900487 non-null object
review_date          6900562 non-null datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int16(2), int8(1), object(8)
memory usage: 572.5+ MB
None
  customer_id       review_id  product_id product_parent  \
0    53096384   R63J84G1LOX6R  1563890119      763187671   
1    53096399  R1BALOA11Z06MT

In [109]:
#Save a sample of the table to upload to github, as the original file is far to large to be uploaded.
reviews.sample(n=10000, random_state=42).to_csv(r'amazon_reviews_sample.csv', index=False)

In [110]:
#Satistics of review ratings
print(reviews[["star_rating","helpful_votes","total_votes"]].describe())

        star_rating  helpful_votes   total_votes
count  6.900562e+06   6.900562e+06  6.900562e+06
mean   4.306589e+00   2.044490e+00  3.251608e+00
std    1.146197e+00   3.184562e+01  3.634051e+01
min    1.000000e+00   0.000000e+00  0.000000e+00
25%    4.000000e+00   0.000000e+00  0.000000e+00
50%    5.000000e+00   0.000000e+00  0.000000e+00
75%    5.000000e+00   1.000000e+00  2.000000e+00
max    5.000000e+00   2.755000e+04  2.872700e+04


In [111]:
print("Number of unique customers:", len(reviews.customer_id.unique()))
print("Number of unique goods:", len(reviews.product_parent.unique()))
print("Number of unique catergories:", len(reviews.product_category.unique()))

Number of unique customers: 4097672
Number of unique goods: 16309
Number of unique catergories: 38


In [112]:
x=reviews.customer_id.value_counts()
y=x[x >5].index.tolist()
reviews.loc[reviews['customer_id'].isin(y)]

,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
34,53090280,R1R1V2QRQ87Q92,0440178002,191357398,Shogun (Asian Saga),Books,5,1,1,False,False,"Excellent historical fiction, well written",It really happened: only the names have been c...,1996-06-26
52,52977241,RULWKAINTXAUR,0393314804,232683594,Trainspotting,Books,5,0,0,False,False,Surreal,"So I cheated. I haven't read the book, but I ...",1996-07-25
55,53090280,R200N7BA6G2YXT,0440178002,191357398,Shogun (Asian Saga),Books,5,2,2,False,False,The Real John Blackthorne,"Like Toranaga, Ishido, Mariko, and the other m...",1996-07-27
69,53093818,R13OOAVKRQIFSO,0812548094,939049888,Stone of Tears,Books,5,0,1,False,False,Even better than the first one,"This book made me a fan of Terry Goodkind, if ...",1996-08-12
94,52748904,R1F2IGEGT8M6T4,0553346687,33679552,Surely You're Joking Mr. Feynman,Books,5,4,5,False,False,Absolutely delightful!,I was checking on this book because I just rec...,1996-09-19
184,53051790,RXI5DPIRSBKVF,0812550706,401439625,Ender's Game (The Ender Quintet),Books,5,5,5,False,False,Xenophobic world creates its greatest leader.....,Ender's Game is the first (and best) in a seri...,1996-12-17
212,52636801,R3PQQG9U109PP1,0312857063,939049888,"Stone of Tears (Sword of Truth, Book 2)",Books,4,1,3,False,False,"Goodkind is a very, very good author, but no r...","I loved this boook, althouhg the story was too...",1997-01-07
315,52977842,R28F1BBT2O57KQ,0440213525,282981210,The Client,Books,2,0,1,False,False,"FAST PACED, BUT A DUMB STORY",THE WEAKEST OF ALL GRISHAM'S BOOKS. I STILL C...,1997-02-17
490,52885970,R3638HA7J3999A,0345315227,284653366,3001: The Final Odyssey,Books,5,1,2,False,False,A must read for all Odyssey series followers,It's an excellent book and a moving final to t...,1997-04-07
589,53067292,R1HZE2HEHGVHRR,044651652X,879659950,The Bridges of Madison County,Books,1,1,2,False,False,"Too bad you can't rate it a big, fat zero!",This is the worst use of typed words in existe...,1997-04-27


In [113]:
reviews.groupby('customer_id').count() 

,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
customer_id,,,,,,,,,,,,,
100000,1,1,1,1,1,1,1,1,1,1,1,1,1
10000030,1,1,1,1,1,1,1,1,1,1,1,1,1
10000031,1,1,1,1,1,1,1,1,1,1,1,1,1
10000037,3,3,3,3,3,3,3,3,3,3,3,3,3
10000040,1,1,1,1,1,1,1,1,1,1,1,1,1
10000049,1,1,1,1,1,1,1,1,1,1,1,1,1
10000059,1,1,1,1,1,1,1,1,1,1,1,1,1
10000064,2,2,2,2,2,2,2,2,2,2,2,2,2
10000087,2,2,2,2,2,2,2,2,2,2,2,2,2


In [115]:
sum(reviews.verified_purchase)

5032988

### What is the structure of your dataset?

> The cleaned dataset contains 6,900,562 unique reviews with 14 features (customer_id, review_id, product_id, product_parent, product_title, product_category, star_rating, helpful_votes, total_votes, vine, verified_purchase, review_headline, review_body, review_date).    
Most of the columns are objects. Some are unique identifiers to either a product, category or the review (customer_id, review_id, product_id); some are strings relating to the the product such as the name of the items, its category or the review contents (product_title, product_category, review_headline, review_body).  
The ratings of each review along with the helpfullness of each review are intergers (star_rating, helpful_votes, total_votes).  
Two columns are boolian as they are either Y(yes) or N(no) (vine, verifited_purchase).  
The date of the review is in datetime format (review_date).  


### What are the main features of interest in your dataset?

> There are a few aspects, I am interested in investigating.   
I'm interested in seeing if there is a relationship between vine reviewers and non-vine reviewers - do they make a difference?
I also want to see if certian people are more likly to leave good reviews only, bad reviews only, or do they cover both aspects?  
Is there any relationship between the contents of a review and the number of helpful votes?  
Are helpful votes only on good or bad reviews?
Do certian words in a title make for a more helpful review?  
Is there are relationship between the date and types of reviews given?

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> I am expecting that the review rating will be one of the most prominent features to be used. Regarding the contents of a review and the number of helpful votes, the review title and review body. For this I am expecting that long reviews will be more helpful to people over shorter revews.

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!